In [147]:
import json
f = open("cmd_setumei.json")
setumei = json.load(f)

In [153]:
n = 3
state = []

def create_state(cmd_row):
    cmd = []
    index = 0
    for item in cmd_row:
        if item[0] != "[":
            cmd.append({
                "val":item,
                "args":[]
            })
            index += 1
        else:
            cmd[index-1]["args"].append({
                "placeholder":item
            })
        
    
    for i in range(len(cmd)):
        s = []
        for j in range(i,min(i+n,len(cmd))):
            s.append(cmd[j])
        
        s = [v for v in s if len(v) >= 2]
        if len(s) >= 2:
            state.append(s)

with open("cmd_data.txt","r") as f:
    for row in f:
        cmd = row.split()
        create_state(cmd)

[{'val': 'git', 'args': []}, {'val': 'add', 'args': [{'placeholder': '[PATH]'}]}]
[{'val': 'git', 'args': []}, {'val': 'push', 'args': []}, {'val': 'origin', 'args': []}, {'val': 'master', 'args': []}]
[{'val': 'git', 'args': []}, {'val': 'commit', 'args': []}, {'val': '-m', 'args': [{'placeholder': '[String]'}]}]
[{'val': 'git', 'args': []}, {'val': 'push', 'args': []}, {'val': 'origin', 'args': [{'placeholder': '[ブランチ名]'}]}]
[{'val': 'git', 'args': []}, {'val': 'diff', 'args': [{'placeholder': '[PATH]'}]}]
[{'val': 'git', 'args': []}, {'val': 'log', 'args': []}]
[{'val': 'find', 'args': [{'placeholder': '[PATH]'}]}, {'val': '-name', 'args': [{'placeholder': '[PATH]'}]}]
[{'val': 'find', 'args': [{'placeholder': '[PATH]'}]}, {'val': '-name', 'args': [{'placeholder': '[PATH]'}]}, {'val': '|', 'args': []}, {'val': 'xargs', 'args': []}, {'val': 'cat', 'args': []}]
[{'val': 'cat', 'args': [{'placeholder': '[PATH]'}]}, {'val': '>>', 'args': [{'placeholder': '[PATH]'}]}]
[{'val': 'cat', 'ar

In [177]:
ng = {}

def create_ngram(states):
    
    for cmd in states:
        path = ng
        for i in range(len(cmd)):
            if cmd[i]['val'] in path:
                path[cmd[i]['val']]["count"] += 1
            else:
                
                if cmd[i]['val'][0] == "-":
                    cmd_type = "option"
                else:
                    cmd_type = "command"
                print(cmd[i]['val'])
                if cmd[i]['val'] in setumei:
                    description = setumei[cmd[i]['val']]
                else:
                    print("target : ",cmd[i])
                    description = input()
                    setumei[cmd[i]['val']] = description
                
                path[cmd[i]['val']] = {
                    "type":cmd_type,
                    "count":1,
                    "description":description,
                    "cmd":cmd[i],
                    "next":{}
                }

            path = path[cmd[i]['val']]["next"]
        



In [178]:
ng

{}

In [179]:
create_ngram(state)

git
add
push
origin
push
origin
master
origin
master
commit
-m
commit
-m
diff
log
find
-name
|
-name
|
xargs
|
xargs
cat
xargs
cat
cat
>>
>
|
grep
grep


In [182]:
import json
model = json.dumps(ng,ensure_ascii=False,indent=2)
with open("model.json","w") as f:
    f.write(model)

In [181]:
import json
des = json.dumps(setumei,ensure_ascii=False,indent=2)
with open("cmd_setumei.json","w") as f:
    f.write(des)

In [183]:
state

[[{'val': 'git', 'args': []},
  {'val': 'add', 'args': [{'placeholder': '[PATH]'}]}],
 [{'val': 'git', 'args': []},
  {'val': 'push', 'args': []},
  {'val': 'origin', 'args': []}],
 [{'val': 'push', 'args': []},
  {'val': 'origin', 'args': []},
  {'val': 'master', 'args': []}],
 [{'val': 'origin', 'args': []}, {'val': 'master', 'args': []}],
 [{'val': 'git', 'args': []},
  {'val': 'commit', 'args': []},
  {'val': '-m', 'args': [{'placeholder': '[String]'}]}],
 [{'val': 'commit', 'args': []},
  {'val': '-m', 'args': [{'placeholder': '[String]'}]}],
 [{'val': 'git', 'args': []},
  {'val': 'push', 'args': []},
  {'val': 'origin', 'args': [{'placeholder': '[ブランチ名]'}]}],
 [{'val': 'push', 'args': []},
  {'val': 'origin', 'args': [{'placeholder': '[ブランチ名]'}]}],
 [{'val': 'git', 'args': []},
  {'val': 'diff', 'args': [{'placeholder': '[PATH]'}]}],
 [{'val': 'git', 'args': []}, {'val': 'log', 'args': []}],
 [{'val': 'find', 'args': [{'placeholder': '[PATH]'}]},
  {'val': '-name', 'args': [{'pl